## 1) Installation des librairies

In [26]:
!pip install langchain langchain-mistralai pydantic



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2) Définir la clé Mistral

In [27]:
import os

os.environ["MISTRAL_API_KEY"] = "KoMaytgiPsamEfcg4wDBPXN6dp20fYaC"



## 3) Imports pour Mistral

In [28]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


## 4) Schéma de sortie

In [29]:
class MusicRecommendation(BaseModel):
    genre: str = Field(description="Genre musical ciblé")
    summary: str = Field(description="Résumé global du style musical conseillé")
    key_characteristics: List[str] = Field(description="Caractéristiques audio clés")
    tempo_guideline: str = Field(description="Indication sur le tempo")
    mood: str = Field(description="Ambiance générale de la musique")


## 5) Parser

In [30]:
parser = PydanticOutputParser(pydantic_object=MusicRecommendation)


## 6) Prompt

In [31]:
prompt = PromptTemplate(
    template="""
Tu es un expert en production musicale et analyse de données Spotify.

Voici des paramètres audio optimaux obtenus par un modèle de machine learning
pour maximiser la popularité d'une musique.

Genre : {genre}

Paramètres audio :
{parameters}

À partir UNIQUEMENT de ces données, génère une recommandation musicale claire,
structurée et exploitable par un humain.

{format_instructions}
""",
    input_variables=["genre", "parameters"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)


## 7) Initialiser Mistral

In [32]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0.3
)


In [33]:
import pandas as pd

top_pop_recipe = pd.read_csv(
    "top_pop_recipe.csv",
    index_col=0
).iloc[:, 0].to_dict()

top_pop_recipe


{'danceability': 0.6104664976343452,
 'energy': 0.6607609583129278,
 'loudness': -6.731318170523909,
 'speechiness': 0.052192985404576,
 'acousticness': 0.3500728041848407,
 'instrumentalness': 0.002839068079716,
 'liveness': 0.1598883663950482,
 'valence': 0.4247178685179048,
 'tempo': 110.97349793497328,
 'duration_ms': 227589.59172928703}

## 8) Appeler LLM

In [34]:
prompt_filled = prompt.format(
    genre="Pop",
    parameters=top_pop_recipe
)

raw_output = llm.invoke(prompt_filled)
recommendation = parser.parse(raw_output.content)

recommendation


MusicRecommendation(genre='Pop', summary="Une production pop moderne et équilibrée, conçue pour maximiser l'engagement et la popularité sur les plateformes de streaming. Ce style combine des éléments dansants, une énergie accessible et une touche d'authenticité acoustique pour créer un son à la fois actuel et intemporel. La structure est optimisée pour capter l'attention dès les premières secondes tout en maintenant une progression dynamique jusqu'à la fin.", key_characteristics=['Équilibre entre énergie et accessibilité : une danceability modérée (0.61) et une énergie marquée (0.66) pour un groove entraînant sans être agressif', 'Production soignée avec un volume maîtrisé (-6.73 dB) pour une écoute confortable sur tous les supports (casque, enceinte, voiture)', "Présence vocale dominante avec très peu d'éléments parlés (speechiness à 0.052) et quasi-absence d'instrumental (0.0028)", "Mélange subtil d'éléments électroniques et acoustiques (acousticness à 0.35) pour une touche organique

## 10) Nouveau schéma pour l'intention utilisateur

In [35]:
from pydantic import BaseModel, Field
from typing import List, Optional

class UserMusicIntent(BaseModel):
    """Ce que l'utilisateur veut en langage humain, mais structuré."""
    description: str = Field(description="Description brute fournie par l'utilisateur")
    genre: Optional[str] = Field(description="Genre musical souhaité (pop, rock, techno, etc.)", default=None)
    energy: Optional[str] = Field(description="Niveau d'énergie souhaité (faible, modérée, élevée)", default=None)
    mood: Optional[List[str]] = Field(description="Liste de mots décrivant l'ambiance (chill, joyeux, sombre, nostalgique...)", default=None)
    danceability: Optional[str] = Field(description="Niveau de côté dansant (faible, moyen, élevé)", default=None)
    tempo: Optional[str] = Field(description="Perception du tempo (lent, moyen, rapide)", default=None)
    duration_pref: Optional[str] = Field(description="Préférence de durée (court, moyen, long)", default=None)
    context: Optional[str] = Field(description="Contexte d'écoute (soirée, voiture, étude, sport, etc.)", default=None)


## 10.1) Création d'un parser + prompt d'extraction d'intentions

In [36]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

intent_parser = PydanticOutputParser(pydantic_object=UserMusicIntent)

intent_prompt = PromptTemplate(
    template="""
Tu es un assistant qui analyse une demande de musique, écrite en français,
et tu dois en extraire des informations structurées.

Voici la demande de l'utilisateur :

\"\"\"{user_text}\"\"\"

Tu dois remplir les champs suivants :
- genre (si possible)
- energy (faible, modérée, élevée)
- mood (liste de mots)
- danceability (faible, moyenne, élevée)
- tempo (lent, moyen, rapide)
- duration_pref (court, moyen, long)
- context (par ex. soirée, voiture, travail, sport...)
- description (reprise ou reformulation du texte d'origine)

{format_instructions}
""",
    input_variables=["user_text"],
    partial_variables={
        "format_instructions": intent_parser.get_format_instructions()
    }
)


## 10.2) Test avec un prompt utilisateur

In [37]:
user_text = """
Je veux une musique pop assez énergique, dansante, joyeuse,
pas trop longue, pour écouter en voiture l'été.
"""

intent_prompt_filled = intent_prompt.format(user_text=user_text)

raw_intent = llm.invoke(intent_prompt_filled)
print(raw_intent.content)

user_intent = intent_parser.parse(raw_intent.content)
user_intent


Voici le JSON structuré conforme à la demande de l'utilisateur :

```json
{
  "description": "Je veux une musique pop assez énergique, dansante, joyeuse, pas trop longue, pour écouter en voiture l'été.",
  "genre": "pop",
  "energy": "élevée",
  "mood": ["joyeuse", "énergique", "estivale"],
  "danceability": "élevée",
  "tempo": "rapide",
  "duration_pref": "court",
  "context": "voiture"
}
```


UserMusicIntent(description="Je veux une musique pop assez énergique, dansante, joyeuse, pas trop longue, pour écouter en voiture l'été.", genre='pop', energy='élevée', mood=['joyeuse', 'énergique', 'estivale'], danceability='élevée', tempo='rapide', duration_pref='court', context='voiture')

## 10.4) Traduire l'intention en paramètres

In [38]:
import numpy as np
import copy

def intent_to_audio_params(user_intent: UserMusicIntent, base_recipe: dict):
    """
    Prend une intention utilisateur + une recette de base (ex: top_pop_recipe)
    et renvoie un dictionnaire de paramètres audio ajustés.
    """
    params = copy.deepcopy(base_recipe)

    # 1) Energy → feature "energy"
    if user_intent.energy == "faible":
        params["energy"] = max(0.1, params["energy"] - 0.2)
    elif user_intent.energy == "élevée":
        params["energy"] = min(0.95, params["energy"] + 0.2)

    # 2) Danceability
    if user_intent.danceability == "faible":
        params["danceability"] = max(0.1, params["danceability"] - 0.2)
    elif user_intent.danceability == "élevée":
        params["danceability"] = min(0.95, params["danceability"] + 0.2)

    # 3) Tempo
    if user_intent.tempo == "lent":
        params["tempo"] = max(70, params["tempo"] - 20)
    elif user_intent.tempo == "rapide":
        params["tempo"] = min(140, params["tempo"] + 15)

    # 4) Valence (humeur globale)
    if user_intent.mood:
        if any(m in user_intent.mood for m in ["joyeux", "festif", "positif"]):
            params["valence"] = min(0.9, params.get("valence", 0.5) + 0.2)
        if any(m in user_intent.mood for m in ["sombre", "triste", "nostalgique"]):
            params["valence"] = max(0.1, params.get("valence", 0.5) - 0.2)

    # 5) Durée
    if user_intent.duration_pref == "court":
        params["duration_ms"] = 160_000  # ~2min40
    elif user_intent.duration_pref == "long":
        params["duration_ms"] = 260_000  # ~4min20

    # 6) Contexte "chill" → baisse un peu energy et tempo
    if user_intent.context and "chill" in user_intent.context:
        params["energy"] = max(0.1, params["energy"] - 0.1)
        params["tempo"] = max(80, params["tempo"] - 10)

    return params


## 6) Générer une réponse en langage naturel

In [40]:
from pprint import pprint

pprint(audio_params_from_intent)


{'acousticness': 0.3500728041848407,
 'danceability': 0.8104664976343452,
 'duration_ms': 160000,
 'energy': 0.8607609583129279,
 'instrumentalness': 0.002839068079716,
 'liveness': 0.1598883663950482,
 'loudness': -6.731318170523909,
 'speechiness': 0.052192985404576,
 'tempo': 125.97349793497328,
 'valence': 0.4247178685179048}


In [41]:
prompt_filled_for_user = prompt.format(
    genre=user_intent.genre or "Pop",
    parameters=audio_params_from_intent
)

raw_output_for_user = llm.invoke(prompt_filled_for_user)
personalized_reco = parser.parse(raw_output_for_user.content)
personalized_reco


MusicRecommendation(genre='pop', summary="Une production pop énergique et dansante, conçue pour maximiser l'engagement et la popularité sur les plateformes de streaming. Ce style combine des éléments modernes et accessibles, avec une touche d'authenticité acoustique pour équilibrer l'intensité électronique. Idéal pour les playlists virales et les moments de fête ou d'entraînement.", key_characteristics=['Rythme très dansant (danceability élevée) : structure rythmique claire et répétitive, idéale pour les mouvements et les chorégraphies', "Énergie élevée : instrumentation dynamique, percussions marquées et synthés puissants pour capter l'attention", 'Volume percutant (loudness à -6.73 dB) : mixage moderne et compétitif, sans saturation excessive', 'Présence vocale dominante (speechiness faible) : les paroles doivent être intelligibles et mélodiques, avec peu de passages parlés', "Équilibre acoustique (35% d'acousticness) : intégration subtile d'éléments organiques (guitares, pianos) pou